<a href="https://colab.research.google.com/github/karlmaji/pytorch_learning/blob/master/Swin_transformer%E8%AE%BA%E6%96%87%E5%A4%8D%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 难点1 img to patch

In [16]:
# patch_depth = patchsize *patchsize *input_channel
# patch_depth= 12
# model_dim = 8
# weight = torch.randn(12,8)


# 方法1：使用unfold 方法
def img2patch(img,patchsize,weight):
  patch = F.unfold(img,kernel_size=patchsize,
                   stride=patchsize).transpose(-1,-2) 
                   #[bs,patch_num,patch_depth]

  patch_embedding = patch @ weight #weight.shape:[patch_depth,model_dim]

  # print(patch_embedding.shape) #[bs,num_patch,model_dim]
  
  return patch_embedding

# 方法2:使用卷积方法
def img2patch_conv(img,patchsize,model_dim):
  bs,i_c,h,w =img.shape

  #实际建立model时 先定义卷积层在调用
  layer= nn.Conv2d(3,model_dim,kernel_size=patchsize,stride=patchsize)
  #output:[bs,model_dim,h,w]

  patch_embedding=layer(img).reshape(bs,model_dim,-1).transpose(-1,-2)
  # print(patch_embedding.shape)


# img=torch.randn(2,3,224,224)
# img2patch(img,2,weight)
# img2patch_conv(img,2,model_dim)
